# Final Project

Our idea is to measure whether there is correlation between the popularity of a country during the time World Cup 2022 is held on twitter and the sales from merchendise of that country. 

### Retrieving and storing Twitter data

First, we will import the Tweepy API, which we can use to retrieve tweets. We created an app which let us create keys and tokens. 

In [2]:
import tweepy

CONSUMER_KEY = 'GAwaUDnN1TOjhb830Z96bOiNS'
CONSUMER_SECRET = 'Yyqst9NDX8U7iaCCyWQ1JdQ27FU9RoQXQJKCQl6M7WzrIiYc7E'
OAUTH_TOKEN = '1364492577957965825-FeM77E1DgijgzOm08Tt7EzGwYuotEa'
OAUTH_TOKEN_SECRET = '66N00O85na9KPdS6UU8iXzpE7eVokFgFA6pt1idI1Txbt'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAK8ojAEAAAAAIZA72KVbwvWMNIo23dF1l7Zry%2BI%3D0W5vM70hjQiYO6pmUg4naYme501nhySYvrDbnEEIWeyGOL3hDC'

client = tweepy.Client(bearer_token)

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

Then we retrieve tweets which contain the hashtag WorldCup. We do not want to have retweets in our dataset, so we use 'is:retweet' to prevent this. To make sure we can maintain access to our tweets, we save them in a textfile. Therefore we can skip the next cell and instead retrieve the tweets from the file we created.

In [3]:
import json 

#query = '#WorldCup, -is:retweet'

#tweets = tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=100000)

#list_of_tweets = []
#for tweet in tweets:
#    list_of_tweets.append(tweet.text)
    
#with open('tweets_worldcup.txt', 'w') as file:
#        file.write(json.dumps(all_tweets, indent=4))

In [28]:
with open('tweets_worldcup.txt', encoding='utf-8') as file:  
    twitter_data = json.load(file)
print(twitter_data[:5])

['STAR MAN SAKA‼\n\n#WorldCup #England #Iran #Saka https://t.co/2tTe7C4YXa', '@CryptoSurfsUp @stripe @QatarGrow meme token supporting the WorldCup vibes with dashboard beating join now \n\n🟢 0% tax BUY\n🟢 CMC/CG AVE\n🟢 Experienced team \n🟢 SAFU \n\nhttps://t.co/WexLh1nMxa\n\nhttps://t.co/DcigpKCtSS\n#qatargrow #worldcup #fifa22 https://t.co/eO5jXLteYQ', '#قطر  #كاس_العالم  #سعوديه  #السعودية  #الأخضر  #الأرجنتين🇦🇷  #saudi  #saudiarabia  #worldcup  #football \n\n#قطر  #كاس_العالم  #سعوديه  #السعودية  #الأخضر  #الأرجنتين🇦🇷  #saudi  #saudiarabia  #worldcup  #football  ##startanonlinesidehustle2020 https://t.co/hc4W4KsnlX', "✅Bet £5 on World Cup\n✅GET £20 in FREE bets to use on anything\n\nCLAIM HERE https://t.co/VHi21J8pDX\nT&amp;C's Apply, New Customers, Play Safe #Ad 18+ #WorldCup https://t.co/eHIlhPofsi", '@gilabola_ina GAS LIVE STREAMING BOLA NOBAR DENMARK VS TUNISIA\n\nhttps://t.co/RHKa9nEPPw\n\n #WorldCup #PialaDunia2022 #PialaDunia2022live #WorldCup2022 #WorldcupQatar2022 #Argentin

We collected 100000 tweets containing the hashtag WorldCup 

In [6]:
print(len(twitter_data))

100000


### Cleaning Twitter data

To clean the twitter data we perform multiple steps. We remove links from the tweets, we remove punctuation, tokenize the tweet and remove stopwords.

In [29]:
import string
import nltk
stopword = nltk.corpus.stopwords.words('english')

def remove_link(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    return tweet

def remove_punct(tweet):
    tweet  = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)
    return tweet

def tokenization(tweet):
    tweet = re.split('\W+', tweet)
    return tweet

def remove_stopwords(tweet):
    tweet = [word for word in tweet if word not in stopword]
    return tweet

In [32]:
import re
cleaned_tweets = []
for tweet in twitter_data[:5]:
    tweet = remove_stopwords(tokenization(remove_punct(remove_link(tweet))))
    tweet = list(filter(None, tweet))
    cleaned_tweets.append(tweet)

cleaned_tweets[:1]

[['STAR', 'MAN', 'SAKA', 'WorldCup', 'England', 'Iran', 'Saka']]

To Do
- create a wordcloud
- measure percentages of languages used in the tweets

### Measuring popularity teams

Our plan for measuring the popularity of teams, will be by collecting a set of keywords per team. For example keywords for the Netherlands could be 'Netherlands', 'Memphis Depay', 'Depay', 'NED', etc. We will count the mentions of these keywords in the tweets containing the hashtag #WorldCup. Furthermore, we will establish the sentiment of the tweet, to figure out whether the mention of the keyword is positive, neutral, or negative. Based on this we will use a weight to calculate the actual popularity of the mention. 

To create a database of keywords, we would like to use a dataset of all players for each country for the World Cup 2022. Since there is no such dataset yet for 2022 we decided to collect this data ourselves. 

In [52]:
import pandas as pd
import sqlite3
df_team_players = pd.read_excel('db_team_players.xlsx')
df_team_players[:5]

,Team,Player,abbreviation
0,Ecuador,Hernán Galíndez,ECU
1,Ecuador,Félix Torres,ECU
2,Ecuador,Piero Hincapié,ECU
3,Ecuador,Robert Arboleda,ECU
4,Ecuador,José Cifuentes,ECU


Players are often reffered to their last name, so we make a column with their last names

In [53]:
df_team_players['lastname'] = df_team_players['Player'].str.split().str[-1]
df_team_players[:5]

,Team,Player,abbreviation,lastname
0,Ecuador,Hernán Galíndez,ECU,Galíndez
1,Ecuador,Félix Torres,ECU,Torres
2,Ecuador,Piero Hincapié,ECU,Hincapié
3,Ecuador,Robert Arboleda,ECU,Arboleda
4,Ecuador,José Cifuentes,ECU,Cifuentes


From this dataframe, we create a dictionary, where the key is the country and the values are the players, the country and the FIFA country codes.

In [54]:
teams = df_team_players["Team"].unique()
keywords_per_country = []
for team in teams:
    keywords = []
    players = df_team_players.loc[df_team_players['Team'] == team, 'Player'].tolist()
    players_lastname = df_team_players.loc[df_team_players['Team'] == team, 'lastname'].tolist()
    abbreviation = df_team_players.loc[df_team_players['Team'] == team, 'abbreviation'].unique()
    keywords.extend(players)
    keywords.extend(players_lastname)
    keywords.append(team)
    keywords.extend(abbreviation)
    keywords_per_country.append(keywords)

dic_keywords = dict(zip(teams, keywords_per_country))
